In [2]:
from datetime import datetime, date, timedelta
import variables
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import yfinance as yf
import pandas as pd 
import sys

In [3]:
# get Path to csv
PATH = variables.stocks

Use the function to get stock df from csv

In [4]:
# Get Dataframes from csv
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File is not here" )
        print('Expected file: ' + PATH + ticker + ".csv")
    else: 
        return df


Get list of tickers to loop through later

In [5]:
# Get all stock tickers
files = [x for x in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
# tickers.remove('.ds_Store')
tickers.sort()
len(tickers)

3261

Testing with just the AMD stock
only selecting the first 5 columns

In [6]:
# Read in the old Stocks CSV and just take the original columns 
# (as we'll add the columns again once the new date rows are added)
old_df = get_stock_df_from_csv("AMD")
old_df = old_df.iloc[: , : 5]


Find the start date (day after last recorded day on CSV) and end date (today)

In [43]:
# Find the start (last day in the old stocks CSV + 1)
start_date = (datetime.strptime(old_df.index[-1], "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

# Find the end date (todays date)
end_date = datetime.today().strftime("%Y-%m-%d")

Use the yfinance package to get the stock data between these dates

In [65]:
# Download the latest stock data and drop the Adj Close  
new_df = yf.download("A", start=start_date, end=end_date)
# Drop Adj column so the column numbers match the csv
new_df = new_df.drop(columns="Adj Close")
# reset the index in order to change the dates to object
new_df = new_df.reset_index()
new_df["Date"] = new_df["Date"].apply(lambda x: x.strftime('%Y-%m-%d'))

[*********************100%***********************]  1 of 1 completed


Date       object
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

Reset the index for old_df

In [58]:
old_df = old_df.reset_index()

Append the new df to the old one

In [71]:
final_df = old_df.append(new_df)
final_df.set_index("Date")

,Open,High,Low,Close,Volume
Date,,,,,
2017-02-08,13.210000,13.750000,13.080000,13.560000,75942900
2017-02-09,13.780000,13.890000,13.400000,13.420000,73339900
2017-02-10,13.860000,13.860000,13.250000,13.580000,54579300
2017-02-13,13.700000,13.950000,13.380000,13.490000,57413100
2017-02-14,13.430000,13.490000,13.190000,13.260000,40479100
...,...,...,...,...,...
2022-02-09,142.729996,145.100006,142.149994,144.440002,1216900
2022-02-10,141.559998,144.039993,140.580002,141.660004,1063700
2022-02-11,141.119995,142.509995,136.240005,137.199997,1321900


Import the columns module made earler to run these functions and add columns

In [73]:
import columns_modules as cm

df = cm.add_daily_return_to_df(final_df)
df = cm.add_cum_return_to_df(df)
df = cm.add_bollinger_bands(df)
df = cm.add_Ichimoku(df)



This is all the steps from above but in a loop to apply it to each ticker

In [74]:
count = 0 
for ticker in tickers:
    count += 1
    try: 
        old_df = get_stock_df_from_csv(ticker)
        start_date = (datetime.strptime(old_df.index[-1], "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
        end_date = datetime.today().strftime("%Y-%m-%d")

        new_df = yf.download(ticker, start=start_date, end=end_date)
        new_df = new_df.drop(columns="Adj Close")
        new_df = new_df.reset_index()
        new_df["Date"] = new_df["Date"].apply(lambda x: x.strftime('%Y-%m-%d'))
    
        old_df = old_df.reset_index()       

        final_df = old_df.append(new_df)
        final_df.set_index("Date")

        df = cm.add_daily_return_to_df(final_df)
        df = cm.add_cum_return_to_df(df)
        df = cm.add_bollinger_bands(df)
        df = cm.add_Ichimoku(df)   
        df.to_csv(PATH + ticker + '.csv')

    except Exception as ex:
        print(ex)
    
    print(f'{count}/{len(tickers)}')


[*********************100%***********************]  1 of 1 completed
1/3261
[*********************100%***********************]  1 of 1 completed
2/3261
[*********************100%***********************]  1 of 1 completed
3/3261
[*********************100%***********************]  1 of 1 completed
4/3261
[*********************100%***********************]  1 of 1 completed
5/3261
[*********************100%***********************]  1 of 1 completed
6/3261
[*********************100%***********************]  1 of 1 completed
7/3261
[*********************100%***********************]  1 of 1 completed
8/3261
[*********************100%***********************]  1 of 1 completed
9/3261
[*********************100%***********************]  1 of 1 completed
10/3261
[*********************100%***********************]  1 of 1 completed
11/3261
index -1 is out of bounds for axis 0 with size 0
12/3261
[*********************100%***********************]  1 of 1 completed
13/3261
[*********************100%**

KeyboardInterrupt: 